Get preds over all data

esm based dynamic model (not using static embeds).


Use TF:
*  🇰https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling-tf.ipynb#scrollTo=de8419b5
* Torch based /Trainer example:  https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling.ipynb#scrollTo=49dcba23



* NOte for pytorch training could use trainer maybe, and mixed precision - https://huggingface.co/docs/transformers/v4.18.0/en/performance#fp16-training

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

# # drive.mount("/content/drive/MyDrive/Research/proteins")
# # drive.mount("/content/drive/MyDrive/proteins")

# !pip3 install tensorflow datasets transformers accelerate fair-esm -U #-q --user
# !pip install torch accelerate transformers fair-esm datasets  -U -q

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

* Use the unirpot fasta file I downloaded and uploaded to my drive

`/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_90.fasta`

* Download fasta from: `https://www.uniprot.org/uniref/?query=uniprot:(keyword%3A%22Transmembrane+%5BKW-0812%5D%22+AND+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22)+identity:0.9`

In [10]:
# DATA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/membrane_100_whole_protbert_embed.snappy.parquet"
# DATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/swp_human_viri_all_embed_esm.parquet" ## ESM1B embedding (max len 1022)
# DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-train.csv.gz"## TRAIN
DATA_PATH = "hum_vir_swp-train.csv.gz"## TRAIN
# DATA_PATH ="/kaggle/input/human-virus-proteins/hum_vir_swp-train.csv"

## TEST data:
# TEST_DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-test.csv.gz"## TRAIN
TEST_DATA_PATH = "hum_vir_swp-test.csv.gz"## TEST
# TEST_DATA_PATH = "/kaggle/input/human-virus-proteins/hum_vir_swp-test.csv"
# ## metadata for all reviewed/swissprot human + virus proteins
# METADATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/SWP_human_viruses_all.xlsx"

TARGET_COL = "virus" ## use for filtering data into 1 class

MAX_LEN = 612 # exclude sequences longer than this. (Not merely truncate)

FAST_RUN = False#True

# Embed sequences in a FASTA file

In [11]:
import torch
# import esm
import tensorflow
import tensorflow as tf
from tensorflow.keras.metrics import AUC as tf_auc

import pandas as pd
import numpy as np
import random
from sklearn.preprocessing  import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_predict
# from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
# # from sklearn.experimental import enable_hist_gradient_boosting
# from sklearn.ensemble import HistGradientBoostingClassifier
from tqdm.autonotebook import tqdm

# from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
# from sklearn.decomposition import PCA

from transformers import TrainingArguments, Trainer, logging
# from accelerate import Accelerator
# from torch.utils.data.dataloader import DataLoader
## https://huggingface.co/docs/transformers/perf_train_gpu_one

In [12]:
# df = pd.read_parquet(DATA_PATH) # numpy to pandas
df = pd.read_csv(DATA_PATH,usecols=["Sequence","virus","Length","Cluster name"])
df_test = pd.read_csv(TEST_DATA_PATH,usecols=["Sequence","virus","Length","Cluster name"])
 ## lengths of all the seqs

df

,Sequence,virus,Length,Cluster name
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,0,102,DET1- and DDB1-associated protein 1
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,1,362,Protein MGF 360-19R
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,0,97,Pyrin domain-containing protein 2
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,0,186,Protein FAM9B
4,MADPAAAQNYPLLKLLHSYTPTTPPRPIPKPAPWAPQKPRRQITND...,1,93,Protein E4
...,...,...,...,...
18996,MDPDKQDALNSIENSIYRTAFKLQSVQTLCQLDLIDSSLIQQVLLR...,0,578,Dystrotelin
18997,MLCPWRTANLGLLLILTIFLVAEAEGAAQPNNSLMLQTSKENHALA...,0,348,Cell surface glycoprotein CD200 receptor 1
18998,MCLRFFSPVPGSTSSATNVTMVVSAGPWSSEKAEMNILEINEKLRP...,0,421,Putative neuroblastoma breakpoint family member 7
18999,MASHAGQQHAPAFGQAARASGPTDGRAASRPSHRQGASEARGDPEL...,1,376,Thymidine kinase


In [13]:
df.groupby(["virus"])["Length"].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
virus,,,,,,,,
0,13587.0,413.60,230.28,11.0,230.5,373.0,558.00,1024.0
1,5414.0,338.62,232.85,11.0,146.0,275.0,490.75,1023.0


In [14]:
print(df["virus"].agg(["mean","sum","count"]).round(2))
# df["Length"].describe().round(1)

mean         0.28
sum       5414.00
count    19001.00
Name: virus, dtype: float64


In [15]:
df_test["Length"].describe().round(1)

count    4491.0
mean      393.8
std       233.7
min        18.0
25%       204.0
50%       350.0
75%       538.5
max      1023.0
Name: Length, dtype: float64

In [16]:
df = df.loc[df["Length"]<=MAX_LEN].reset_index(drop=True).copy()
df_test = df_test.loc[df_test["Length"]<=MAX_LEN].reset_index(drop=True).copy()

print(df["Length"].describe().round(1))
print(df["virus"].agg(["mean","sum","count"]).round(2))

count    15535.0
mean       306.0
std        151.7
min         11.0
25%        176.0
50%        302.0
75%        428.0
max        612.0
Name: Length, dtype: float64
mean         0.3
sum       4670.0
count    15535.0
Name: virus, dtype: float64


In [17]:
if FAST_RUN:
    df = df[df["Length"]>450].sample(25).reset_index(drop=True).copy()
    df_test = df_test[df_test["Length"]>=500].sample(17).reset_index(drop=True).copy()

In [18]:
# # ## metadata about all sequences, can be used to identify and to define targets/labels
### Old cleaning/gen  wrangle code: 
# # new : # more correct and coverage ? (by ~200 cases? despite being smaller file)
# df_meta = pd.read_csv("SWP_human_or_virus-animal-data.csv",
#                      usecols=[ 'Protein names', 'Organism', 'Sequence',
#         'Mass',"av_mw", 'Keywords', 'Virus hosts', 'Taxonomic lineage',"UR50_Cluster ID"]) 
# print(df_meta.nunique())
# print(df_meta["UR50_Cluster ID"].isna().mean()) # 16% of cases missing cluster for some reason. Can't use generic "missing" name, so use prot name? First part of entry name?  organism? 
# ## df_meta["Entry Name"].str.split("_",expand=True)[0].nunique() # 20540
# df_meta["group"] = df_meta["UR50_Cluster ID"].fillna(df_meta['Protein names'].str.split("_",expand=True)[0].str.strip())

# # df_meta["av_mw"] = df_meta["Mass"].div(df_meta["Length"]).round(2)
# df_meta["Keywords"] = df_meta["Keywords"].str.replace(";"," . ").str.strip()
# df_meta["Taxonomic lineage"] = df_meta["Taxonomic lineage"].str.replace(" (no rank)","").str.replace(" (superkingdom)","").str.strip()

# display(df_meta)

# df_meta.to_csv("SWP_humVir_meta.csv.gz",index=False,compression="gzip")


In [37]:
df_meta = pd.read_csv("SWP_humVir_meta.csv.gz")
print(df_meta.nunique())
df_meta

Sequence             25119
Protein names        21394
Organism              1136
Mass                 22130
Keywords             15980
av_mw                 2110
Virus hosts            291
Taxonomic lineage      561
UR50_Cluster ID      20886
group                23473
dtype: int64


,Sequence,Protein names,Organism,Mass,Keywords,av_mw,Virus hosts,Taxonomic lineage,UR50_Cluster ID,group
0,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,Putative cytochrome P450 2D7 (EC 1.14.14.1),Homo sapiens (Human),57489,Cytoplasm . Glycoprotein . Heme . Iron . Membr...,111.63,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",NaN,Putative cytochrome P450 2D7 (EC 1.14.14.1)
1,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,Protein PIGBOS1 (PIGB opposite strand protein 1),Homo sapiens (Human),6313,Direct protein sequencing . Membrane . Mitocho...,116.91,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",A0A0B4J2F0,A0A0B4J2F0
2,MRWQEMGYIFYPRKLR,Mitochondrial-derived peptide MOTS-c (Mitochon...,Homo sapiens (Human),2175,DNA-binding . Mitochondrion . Nucleus . Osteog...,135.94,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",A0A0C5B5G6,A0A0C5B5G6
3,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...,Protein CD300H (CD300 antigen-like family memb...,Homo sapiens (Human),21806,Alternative splicing . Disulfide bond . Glycop...,108.49,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",A0A0K2S4Q6,A0A0K2S4Q6
4,MGDQPCASGRSTLPPGNAREAKPPKKRCLLAPRWDYPEGTPNGGST...,Negative regulator of P-body association (P-bo...,Homo sapiens (Human),7025,Cytoplasm . mRNA processing . Reference proteome,103.31,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",A0A0U1RRE5,A0A0U1RRE5
...,...,...,...,...,...,...,...,...,...,...
25114,MKTSPRLSVIDGVGEEVRPDLFQLVHVVPPNTCEPPEQWLRVTVLL...,Uncharacterized protein ORF6,Beak and feather disease virus (BFDV),9846,NaN,111.89,Gracula [TaxID: 116991]; Psittaciformes [TaxID...,"Viruses, Monodnaviria, Shotokuvirae (kingdom),...",Q9YUD1,Q9YUD1
25115,MGGAYITSRYWLVPAHVTSRLRLTSFTGHRGVARVDGPMSSLGLNI...,Uncharacterized protein ORF5,Beak and feather disease virus (BFDV),10863,NaN,107.55,Gracula [TaxID: 116991]; Psittaciformes [TaxID...,"Viruses, Monodnaviria, Shotokuvirae (kingdom),...",Q9YUD2,Q9YUD2
25116,MPYVDFWELSRDFLHFASGHHGSNSSVERAVTVSRHNAQGEYNVPL...,Uncharacterized protein ORF3,Beak and feather disease virus (BFDV),17738,NaN,110.86,Gracula [TaxID: 116991]; Psittaciformes [TaxID...,"Viruses, Monodnaviria, Shotokuvirae (kingdom),...",Q9YUD4,Q9YUD4
25117,MNNGQATITRNGGRFEIRCRHLDRDYTMPLPNATSNDNFLDCIKFI...,Non-structural protein 3 (NS3),Banna virus (strain Indonesia/JKT-6423/1980) (...,34991,Reference proteome,114.35,Bos taurus (Bovine) [TaxID: 9913]; Culex annul...,"Viruses, Riboviria, Orthornavirae (kingdom), D...",Q9YWQ0,Q9YWQ0


### example pretrained fb/torch:
* https://github.com/facebookresearch/esm#getting-started-with-this-repo-

* Transformers + trainer example : (mlm case): https://github.com/facebookresearch/esm/discussions/556
* keras models supported / via HF?
  * https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example


  TF finetuning example (sequence evel?):
  https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling-tf.ipynb#scrollTo=4b26b828

In [19]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AdamWeightDecay
from tensorflow.keras.optimizers import Adafactor, Adam # more memory effecient than adamWD
import tensorflow

## https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example
from transformers import AutoTokenizer, EsmForSequenceClassification, Trainer, TFAutoModel
from transformers import TFAutoModelForTokenClassification, TFAutoModelForSequenceClassification ,TFEsmForSequenceClassification

ImportError: cannot import name 'Adafactor' from 'tensorflow.keras.optimizers' (/Users/oferd2/anaconda3/envs/ot/lib/python3.10/site-packages/keras/api/_v2/keras/optimizers/__init__.py)

In [21]:
train_sequences = df["Sequence"].tolist()
train_labels = df["virus"].tolist()
train_groups = df["Cluster name"].tolist()

# train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)#,stratify=labels)

test_sequences = df_test["Sequence"].tolist()
test_labels = df_test["virus"].tolist()

df_all = pd.concat([df,df_test])

In [22]:
df_all

,Sequence,virus,Length,Cluster name
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,0,102,DET1- and DDB1-associated protein 1
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,1,362,Protein MGF 360-19R
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,0,97,Pyrin domain-containing protein 2
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,0,186,Protein FAM9B
4,MADPAAAQNYPLLKLLHSYTPTTPPRPIPKPAPWAPQKPRRQITND...,1,93,Protein E4
...,...,...,...,...
3677,MAAGDGDVKLGTLGSGSESSNDGGSESPGDAGAAAEGGGWAAAALA...,0,223,Membrane-associated progesterone receptor comp...
3678,MTLPASLKGFETLDIFNISSVRNICELNKHAEHVSIKNLPLIDISV...,1,256,Cytoplasmic envelopment protein 1
3679,MKWKALFTAAILQAQLPITEAQSFGLLDPKLCYLLDGILFIYGVIL...,0,164,T-cell surface glycoprotein CD3 zeta chain
3680,MAGVGDAAAPGEGGGGGVDGPQRDGRGEAEQPGGSGGQGPPPAPQL...,0,567,Transmembrane anterior posterior transformatio...


In [14]:
# # Load ESM-2 model
# ## smallest: esm2_t6_8M_UR50D
# ## 2d smallest
# ## large: esm2_t33_650M_UR50D
# # model_checkpoint =  "facebook/esm2_t12_35M_UR50D"#"facebook/esm2_t6_8M_UR50D"
# model_checkpoint =  "facebook/esm2_t30_150M_UR50D"
#  # ElnaggarLab/ankh-base
#  ### https://github.com/agemagician/Ankh/blob/main/examples/binary_classification_solubility_task.ipynb - different model?
#  #  https://github.com/agemagician/Ankh#models   - 450M model size
# # model_checkpoint =   "ElnaggarLab/ankh-base"

# model_max_len = min(800,MAX_LEN)

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding=True, truncation=True,max_length=model_max_len)

# num_labels = max(train_labels + test_labels) #+ 1  # Add 1 since 0 can be a label
# print("Num labels:", num_labels)
# model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification") # worked, orig
# # model = EsmForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
# # model = TFEsmForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification")

# # model.eval()  # disables dropout for deterministic results # from original - in infer/embed only example

# # last_layer_num = model.num_layers ## 33 for esm2_t33_650M_UR50D


In [15]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalMaxPooling1D, GlobalAveragePooling1D, Concatenate, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.models import Model


# # Load pre-trained model and tokenizer
model_checkpoint = "facebook/esm2_t30_150M_UR50D"#"facebook/esm2_t6_8M_UR50D"  # Replace with your pre-trained model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# # Parameters
model_max_len = MAX_LEN
batch_size_train = 4#5
batch_size_test = 4#5
learning_rate = 1e-4 #8e-5
num_labels = 1  # Replace with the number of unique labels in your dataset

# # Tokenize the sequences
# train_tokenized = tokenizer(train_sequences, padding=True, truncation=True, max_length=model_max_len)
# test_tokenized = tokenizer(test_sequences, padding=True, truncation=True, max_length=model_max_len)

# # Convert to TensorFlow Dataset
# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_tokenized),
#     train_labels
# )).batch(batch_size_train).shuffle(buffer_size=len(train_sequences))

# test_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(test_tokenized),
#     test_labels
# )).batch(batch_size_test)

# ## added. (Awhat about add col labels?)
# train_dataset = train_dataset.map(lambda x, y: ({'input_layer': x['input_ids'], 'attention_mask': x['attention_mask']}, y))
# test_dataset = test_dataset.map(lambda x, y: ({'input_layer': x['input_ids'], 'attention_mask': x['attention_mask']}, y))

# # Load the pre-trained transformer model
# base_model = TFAutoModel.from_pretrained(model_checkpoint)

# # Freeze all layers except the last few
# for layer in base_model.layers[:-4]: #[1:-3]:
#     layer.trainable = False

# # Create new model on top
# input_layer = Input(shape=(model_max_len,), dtype=tf.int32, name='input_layer')
# sequence_output = base_model(input_layer)[0]
# sequence_output = SpatialDropout1D(0.15)(sequence_output)
# # Add GlobalMeanPooling1D and GlobalMaxPooling1D
# avg_pool = GlobalAveragePooling1D()(sequence_output)
# # avg_pool = Dropout(0.15)(avg_pool)
# max_pool = GlobalMaxPooling1D()(sequence_output)
# # max_pool = Dropout(0.15)(max_pool)
# concat = Concatenate()([avg_pool, max_pool])
# concat = Dropout(0.1)(concat)

# # # Initialize mixed precision training
# # from tensorflow.keras import mixed_precision
# # policy = mixed_precision.Policy('mixed_float16')
# # mixed_precision.set_global_policy(policy)


# # Add output layer for binary classification
# output = Dense(1, activation='sigmoid', dtype='float32')(concat)  # Ensuring the dtype is float32 for the output layer

# # Compile model
# final_model = Model(inputs=input_layer, outputs=output)
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# loss = "BinaryFocalCrossentropy"#tf.keras.losses.BinaryCrossentropy(from_logits=False)

# final_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy",tf.keras.metrics.AUC(num_thresholds=12,from_logits=True)])

# # Train the model
# final_model.fit(train_dataset, validation_data=test_dataset, epochs=4)


### CV all points version (Slower!)

In [16]:
assert len(train_sequences+test_sequences) == len(train_labels+test_labels)
assert len(train_labels+test_labels)>len(train_labels)

In [17]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import tensorflow as tf

# Let's assume `all_sequences` and `all_labels` contain all your data points, including train and test.
# 'groups' contains group labels for each data point to ensure stratification by groups.

n_splits = 4  # Number of folds
skf = StratifiedKFold(n_splits=n_splits)

all_sequences = df_all["Sequence"].tolist()
all_labels = df_all["virus"].tolist()
groups = df_all["Cluster name"].tolist()

all_sequences = np.array(all_sequences)
all_labels = np.array(all_labels)


# Placeholder for predictions
all_predictions = np.zeros(len(all_labels))

# Loop through each fold
for train_index, test_index in skf.split(all_sequences, all_labels, groups):
    print("\n\n FOLD \n\n")
#     train_sequences, test_sequences = all_sequences[train_index], all_sequences[test_index]
    train_labels, test_labels = all_labels[train_index], all_labels[test_index]
    train_sequences = [all_sequences[i] for i in train_index]
    test_sequences = [all_sequences[i] for i in test_index]

    # Tokenize and prepare datasets
    train_tokenized = tokenizer(train_sequences, padding=True, truncation=True, max_length=model_max_len)
    test_tokenized = tokenizer(test_sequences, padding=True, truncation=True, max_length=model_max_len)

    train_dataset = tf.data.Dataset.from_tensor_slices((
        dict(train_tokenized),
        train_labels
    )).batch(batch_size_train).shuffle(buffer_size=len(train_sequences))

    test_dataset = tf.data.Dataset.from_tensor_slices((
        dict(test_tokenized),
        test_labels
    )).batch(batch_size_test)
    
    train_dataset = train_dataset.map(lambda x, y: ({'input_layer': x['input_ids'], 'attention_mask': x['attention_mask']}, y))
    test_dataset = test_dataset.map(lambda x, y: ({'input_layer': x['input_ids'], 'attention_mask': x['attention_mask']}, y))

    print("Dataset ready")
    # Load the pre-trained transformer model
    base_model = TFAutoModel.from_pretrained(model_checkpoint)

    # Freeze all layers except the last few
    for layer in base_model.layers[:-8]: #[2:-5]:
        layer.trainable = False

    # Create new model on top
    input_layer = Input(shape=(model_max_len,), dtype=tf.int32, name='input_layer')
    sequence_output = base_model(input_layer)[0]
    sequence_output = SpatialDropout1D(0.1)(sequence_output)
    # Add GlobalMeanPooling1D and GlobalMaxPooling1D
    avg_pool = GlobalAveragePooling1D()(sequence_output)
    max_pool = GlobalMaxPooling1D()(sequence_output)
    concat = Concatenate()([avg_pool, max_pool])
#     concat = Dropout(0.05)(concat)
    
    # Add output layer for binary classification
    output = Dense(1, activation='sigmoid', dtype='float32')(concat)

    # Compile model
    final_model = Model(inputs=input_layer, outputs=output)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = "BinaryFocalCrossentropy"

    final_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy",tf.keras.metrics.AUC(num_thresholds=8,from_logits=True)])

    # Train on the fold
    final_model.fit(train_dataset, epochs=3)#, verbose=0)  # Turn off verbosity if needed

    # Predict on the test fold
    predictions = final_model.predict(test_dataset)
    all_predictions[test_index] = predictions.squeeze()

print("Cross-validation complete. Predictions are stored in 'all_predictions'.")

2023-08-29 14:48:06.767268: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 14:48:06.770109: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 14:48:06.770127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 14:48:06.772242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 14:48:06.772262: I tensorflow/compile

Epoch 1/3


2023-08-29 14:48:23.457979: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [14412]
	 [[{{node Placeholder/_2}}]]
2023-08-29 14:48:23.458135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [14412,612]
	 [[{{node Placeholder/_1}}]]
/home/ddofer/anaconda3/lib/python3.10/site-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['attention_mask'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


2023-08-29 14:48:48.914691: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f0778042c30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-29 14:48:48.914718: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2023-08-29 14:48:48.940393: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-29 14:48:49.126643: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-08-29 14:48:49.261339: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2023-08-29 14:48:49.342976: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3603/3603 [==============================] - 1190s 318ms/step - loss: 0.0712 - accuracy: 0.8773 - auc: 0.8396
Epoch 2/3
3603/3603 [==============================] - 1142s 317ms/step - loss: 0.0418 - accuracy: 0.9371 - auc: 0.9154
Epoch 3/3
3603/3603 [==============================] - 1143s 317ms/step - loss: 0.0251 - accuracy: 0.9656 - auc: 0.9599


...

Epoch 1/3


2023-08-29 17:53:32.560312: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [14413,612]
	 [[{{node Placeholder/_1}}]]
2023-08-29 17:53:32.560453: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [14413,612]
	 [[{{node Placeholder/_1}}]]


3604/3604 [==============================] - 1188s 317ms/step - loss: 0.0756 - accuracy: 0.8666 - auc_3: 0.8256
Epoch 2/3
3604/3604 [==============================] - 1143s 317ms/step - loss: 0.0410 - accuracy: 0.9387 - auc_3: 0.9174
Epoch 3/3
3604/3604 [==============================] - 1151s 319ms/step - loss: 0.0260 - accuracy: 0.9650 - auc_3: 0.9579


2023-08-29 18:51:34.709612: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [4804]
	 [[{{node Placeholder/_2}}]]


1201/1201 [==============================] - 180s 147ms/step
Cross-validation complete. Predictions are stored in 'all_predictions'.


In [18]:
all_predictions.shape

(19217,)

In [19]:
# # model.save("trained_esm.keras")
# model.save("trained_esm_pool.tf",save_format="tf")

In [20]:
# print(classification_report(test_labels,y_test_pred.logits>=0.5))
print(classification_report(df_all["virus"],all_predictions>=0.5))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95     13691
           1       0.90      0.84      0.87      5526

    accuracy                           0.93     19217
   macro avg       0.92      0.90      0.91     19217
weighted avg       0.93      0.93      0.93     19217



In [21]:
# roc_auc_score(test_labels,y_test_pred.logits)
roc_auc_score(df_all["virus"],all_predictions)

0.973929221066181

In [44]:
df_all.drop(columns=["pred_score"],inplace=True)

In [45]:
y_test_binary_preds = (all_predictions>=0.5).astype(int).ravel()

test_mistakes_mask = df_all["virus"]!=y_test_binary_preds
print(test_mistakes_mask.sum())
print(len(test_mistakes_mask))
df_all["mistake"] = test_mistakes_mask

df_all["pred_score_1"] = all_predictions.round(3)

1398
19217


In [46]:
df_all[test_mistakes_mask & (df_all["virus"]==1)].sort_values("pred_score_1",ascending=True)

,Sequence,virus,Length,Cluster name,mistake,pred_score_1
1884,MISWLPSVAMGSRLLCCVALCLLGAGPADSGLTQTPRHLVKARGQQ...,1,321,Viral T-cell receptor beta chain-like T17T-22,True,0.012
1500,MKMAGWKKKLCPGHHLWALGCYMLLAVVSLRLSLRFKCDVDSLDLE...,1,440,"Beta-1,3-galactosyl-O-glycosyl-glycoprotein be...",True,0.012
1949,NRTDVQCQHRWQKVLNPELNKGPWTKEEDQRVIEHVQKYGPKRWSD...,1,382,Transforming protein Myb,True,0.014
2744,MPLSVSLPSKNYDYDYDSVQPYFYFEEEEENFYLAAQQRSSELQPP...,1,412,Viral myc transforming protein,True,0.015
817,MPLSASLPSKNYDYDYDSVQPYFYFEEEEENFYLAAQQRGSELQPP...,1,416,Viral myc transforming protein,True,0.016
...,...,...,...,...,...,...
1598,MELQVVPVDFTAKATSQSLLPLLIALTVCFLSIIILIFVSEFLLYS...,1,94,Probable protein E5,True,0.497
12449,MSLQGIQLSDLSYKHAILKESQYTIKRDVGTTPAVTPSSLQREVSL...,1,394,Nucleoprotein,True,0.497
9485,MMICFVFLCVLTFVRGEIYPSTCPALGAGNGEAVRSGEMLLEISAY...,1,433,U21 glycoprotein,True,0.499
11993,MAFYLPDWSCCGLWLFGRPRNRYSQLPEEPETFECPDRWRAEIDLG...,1,218,Tegument protein UL51 homolog,True,0.499


In [ ]:
df_all.loc[df_all["virus"]==0,"pred_score"]

In [47]:
df_all[test_mistakes_mask & (df_all["virus"]==0)].sort_values("pred_score_1",ascending=False)

,Sequence,virus,Length,Cluster name,mistake,pred_score_1
80,MECLYYFLGFLLLAARLPLDAAKRFHDVLGNERPSAYMREHNQLNG...,0,572,Transmembrane glycoprotein NMB,True,0.933
9662,MEAAAQFFVESPDVVYGPEAIEAQYEYRTTRVSREGGVLKVHPTST...,0,558,Inositol-3-phosphate synthase 1,True,0.926
1705,MHVAEVAVNVILLLSMGWTSDSLCSPTIFYRDCWIRRFPGLLINLE...,0,340,MANSC domain-containing protein 4,True,0.917
9450,MWTVPSFTNDSYQVYNVFSTNSFQLLTVKRTPHEAWRVPLTTKTNK...,0,482,Endogenous retrovirus group K member 13-1 Env ...,True,0.914
2334,MQTLRQEAARPCIPSGTLEASFPAPLYSDDYLSLEGSRWPPAIRQA...,0,209,Uncharacterized protein C19orf71,True,0.913
...,...,...,...,...,...,...
9187,MGVPRTPSRTVLFERERTGLTYRVPSLLPVPPGPTLLAFVEQRLSP...,0,484,Sialidase-4,True,0.503
1348,MADLEEQLSDEEKVRIAAKFIIHAPPGEFNEVFNDVRLLLNNDNLL...,0,286,F-actin-capping protein subunit alpha-2,True,0.503
14436,MGSILSRRIAGVEDIDIQANSAYRYPPKSGNYFASHFFMGGEKFDT...,0,552,E3 ubiquitin-protein ligase MGRN1,True,0.502
7299,MDTQTHSLPITHTQLHSNSQPQSRTCTRHCQTFSQSCRQSHRGSRS...,0,138,Nuclear transition protein 2,True,0.502


In [33]:
df_all[test_mistakes_mask]

,Sequence,virus,Length,Cluster name,mistake,pred_score
7,MNPSEMQRKAPPRRRRHRNRAPSSHKMNKMMMSEEQMKLPSTNKAE...,0,105,Endogenous retrovirus group K member 16 Rec pr...,True,0.7947
9,MEGEKNWIVVPTWRIPGRLEKWHSLVKYLKHRTKELQQVSYVPHHK...,1,216,Virion infectivity factor,True,0.2859
14,MDHGKYLLTIFLNDDDSFFKYLAAQEDDVAMSDVHTIVDYLNFLLA...,1,148,MF7 protein,True,0.3750
19,MFDKIVIPRQFHHQHKWDKLLSKCPHPGVGFYIIGNKKALSARMRT...,1,205,Probable DNA-binding protein,True,0.0309
21,MDYVSLLNQFWQKQIKSYKETPSQYHYLYPPRFFYKPVLGNLQHPT...,1,119,Non-structural protein 3b,True,0.3432
...,...,...,...,...,...,...
3579,MNIRGAPDLGQPSDDPSSGGERERIRQRMKMVIGQLEGILRELKEV...,0,293,Inhibitory synaptic factor 1,True,0.5399
3580,MKSIRQLLSLAKKEKKREKKSNHGSHSMEWESPPSYNEIKSPSAPI...,1,229,Matrix protein,True,0.2237
3594,MSFLGGFFGPICEIDIVLNDGETRKMAEMKTEDGKVEKHYLFYDGE...,0,327,Vacuolar protein sorting-associated protein 26A,True,0.6473
3633,MYKCSQGAMNTEKVMEKFVIQSRFREMYPDKAKAIAGMTVPARYAD...,1,90,Uncharacterized protein 045R,True,0.4516


In [48]:
df_all.to_csv("data_with_mistakes.csv.gz",index=False,compression="gzip")


In [63]:
df_all.drop_duplicates()

,Sequence,virus,Length,Cluster name,mistake,pred_score_1
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,0,102,DET1- and DDB1-associated protein 1,False,0.232
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,1,362,Protein MGF 360-19R,False,0.942
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,0,97,Pyrin domain-containing protein 2,False,0.134
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,0,186,Protein FAM9B,False,0.153
4,MADPAAAQNYPLLKLLHSYTPTTPPRPIPKPAPWAPQKPRRQITND...,1,93,Protein E4,False,0.533
...,...,...,...,...,...,...
3677,MAAGDGDVKLGTLGSGSESSNDGGSESPGDAGAAAEGGGWAAAALA...,0,223,Membrane-associated progesterone receptor comp...,False,0.041
3678,MTLPASLKGFETLDIFNISSVRNICELNKHAEHVSIKNLPLIDISV...,1,256,Cytoplasmic envelopment protein 1,False,0.916
3679,MKWKALFTAAILQAQLPITEAQSFGLLDPKLCYLLDGILFIYGVIL...,0,164,T-cell surface glycoprotein CD3 zeta chain,False,0.089
3680,MAGVGDAAAPGEGGGGGVDGPQRDGRGEAEQPGGSGGQGPPPAPQL...,0,567,Transmembrane anterior posterior transformatio...,False,0.047


In [64]:
df_all.isna().sum()

Sequence        0
virus           0
Length          0
Cluster name    0
mistake         0
pred_score_1    0
dtype: int64

In [68]:
# pd.read_csv("SWP_human_or_virus-animal.tsv",sep="\t",compression="gzip").to_csv("SWP_human_or_virus-animal.csv.gz",index=False,compression="gzip")

In [23]:
df_all

,Sequence,virus,Length,Cluster name
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,0,102,DET1- and DDB1-associated protein 1
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,1,362,Protein MGF 360-19R
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,0,97,Pyrin domain-containing protein 2
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,0,186,Protein FAM9B
4,MADPAAAQNYPLLKLLHSYTPTTPPRPIPKPAPWAPQKPRRQITND...,1,93,Protein E4
...,...,...,...,...
3677,MAAGDGDVKLGTLGSGSESSNDGGSESPGDAGAAAEGGGWAAAALA...,0,223,Membrane-associated progesterone receptor comp...
3678,MTLPASLKGFETLDIFNISSVRNICELNKHAEHVSIKNLPLIDISV...,1,256,Cytoplasmic envelopment protein 1
3679,MKWKALFTAAILQAQLPITEAQSFGLLDPKLCYLLDGILFIYGVIL...,0,164,T-cell surface glycoprotein CD3 zeta chain
3680,MAGVGDAAAPGEGGGGGVDGPQRDGRGEAEQPGGSGGQGPPPAPQL...,0,567,Transmembrane anterior posterior transformatio...


Sequence             25119
Protein names        21394
Organism              1136
Mass                 22130
Keywords             15980
av_mw                 2110
Virus hosts            291
Taxonomic lineage      561
UR50_Cluster ID      20886
dtype: int64
0.16851785501015168


,Sequence,Protein names,Organism,Mass,Keywords,av_mw,Virus hosts,Taxonomic lineage,UR50_Cluster ID,group
0,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,Putative cytochrome P450 2D7 (EC 1.14.14.1),Homo sapiens (Human),57489,Cytoplasm . Glycoprotein . Heme . Iron . Membr...,111.63,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",NaN,Putative cytochrome P450 2D7 (EC 1.14.14.1)
1,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,Protein PIGBOS1 (PIGB opposite strand protein 1),Homo sapiens (Human),6313,Direct protein sequencing . Membrane . Mitocho...,116.91,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",A0A0B4J2F0,A0A0B4J2F0
2,MRWQEMGYIFYPRKLR,Mitochondrial-derived peptide MOTS-c (Mitochon...,Homo sapiens (Human),2175,DNA-binding . Mitochondrion . Nucleus . Osteog...,135.94,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",A0A0C5B5G6,A0A0C5B5G6
3,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...,Protein CD300H (CD300 antigen-like family memb...,Homo sapiens (Human),21806,Alternative splicing . Disulfide bond . Glycop...,108.49,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",A0A0K2S4Q6,A0A0K2S4Q6
4,MGDQPCASGRSTLPPGNAREAKPPKKRCLLAPRWDYPEGTPNGGST...,Negative regulator of P-body association (P-bo...,Homo sapiens (Human),7025,Cytoplasm . mRNA processing . Reference proteome,103.31,NaN,"cellular organisms, Eukaryota, Opisthokonta, M...",A0A0U1RRE5,A0A0U1RRE5
...,...,...,...,...,...,...,...,...,...,...
25114,MKTSPRLSVIDGVGEEVRPDLFQLVHVVPPNTCEPPEQWLRVTVLL...,Uncharacterized protein ORF6,Beak and feather disease virus (BFDV),9846,NaN,111.89,Gracula [TaxID: 116991]; Psittaciformes [TaxID...,"Viruses, Monodnaviria, Shotokuvirae (kingdom),...",Q9YUD1,Q9YUD1
25115,MGGAYITSRYWLVPAHVTSRLRLTSFTGHRGVARVDGPMSSLGLNI...,Uncharacterized protein ORF5,Beak and feather disease virus (BFDV),10863,NaN,107.55,Gracula [TaxID: 116991]; Psittaciformes [TaxID...,"Viruses, Monodnaviria, Shotokuvirae (kingdom),...",Q9YUD2,Q9YUD2
25116,MPYVDFWELSRDFLHFASGHHGSNSSVERAVTVSRHNAQGEYNVPL...,Uncharacterized protein ORF3,Beak and feather disease virus (BFDV),17738,NaN,110.86,Gracula [TaxID: 116991]; Psittaciformes [TaxID...,"Viruses, Monodnaviria, Shotokuvirae (kingdom),...",Q9YUD4,Q9YUD4
25117,MNNGQATITRNGGRFEIRCRHLDRDYTMPLPNATSNDNFLDCIKFI...,Non-structural protein 3 (NS3),Banna virus (strain Indonesia/JKT-6423/1980) (...,34991,Reference proteome,114.35,Bos taurus (Bovine) [TaxID: 9913]; Culex annul...,"Viruses, Riboviria, Orthornavirae (kingdom), D...",Q9YWQ0,Q9YWQ0


In [31]:
df_meta['Protein names'].str.split("_",expand=True)[0].str.strip().nunique() # 21394 - same as protein name

21394

In [28]:
df_meta["Entry Name"].str.split("_",expand=True)[0].nunique() # 20540

20540

In [25]:
df_meta.loc[df_meta["Sequence"].isin(df_all["Sequence"])].shape[0] 

19217

In [67]:
df_merged = df_all.merge(df_meta,on="Sequence",how="left")
# df_merged["av_mw"] = df_merged["Mass"].div(df_merged["Length"]).round(2)
# df_merged["Keywords"] = df_merged["Keywords"].str.replace(";"," . ").str.strip()
# df_merged["Taxonomic lineage"] = df_merged["Taxonomic lineage"].str.replace("(no rank)","").str.replace("(superkingdom)","").str.strip()
print(df_merged.shape)
display(df_merged)
df_merged.to_csv("data_with_mistakes_features_v1.csv.gz",index=False,compression="gzip")

(19606, 13)


,Sequence,virus,Length,Cluster name,mistake,pred_score_1,Protein names,Organism,Mass,Keywords,Virus hosts,Taxonomic lineage,av_mw
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,0,102,DET1- and DDB1-associated protein 1,False,0.232,DET1- and DDB1-associated protein 1 (Placenta ...,Homo sapiens (Human),11835,3D-structure . Acetylation . Phosphoprotein . ...,NaN,"cellular organisms , Eukaryota , Opisthokonta ...",116.03
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,1,362,Protein MGF 360-19R,False,0.942,Protein MGF 360-19R,African swine fever virus (isolate Tick/South ...,42605,ANK repeat,Ornithodoros (relapsing fever ticks) [TaxID: 6...,"Viruses , Varidnaviria , Bamfordvirae (kingdom...",117.69
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,0,97,Pyrin domain-containing protein 2,False,0.134,Pyrin domain-containing protein 2 (Pyrin-only ...,Homo sapiens (Human),10795,Cytoplasm . Immunity . Inflammatory response ....,NaN,"cellular organisms , Eukaryota , Opisthokonta ...",111.29
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,0,186,Protein FAM9B,False,0.153,Protein FAM9B,Homo sapiens (Human),22438,Alternative splicing . Nucleus . Reference pro...,NaN,"cellular organisms , Eukaryota , Opisthokonta ...",120.63
4,MADPAAAQNYPLLKLLHSYTPTTPPRPIPKPAPWAPQKPRRQITND...,1,93,Protein E4,False,0.533,Protein E4,Human papillomavirus 35,10089,Early protein . Host cytoplasm . Host G2/M cel...,Homo sapiens (Human) [TaxID: 9606],"Viruses , Monodnaviria , Shotokuvirae (kingdom...",108.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19601,MAAGDGDVKLGTLGSGSESSNDGGSESPGDAGAAAEGGGWAAAALA...,0,223,Membrane-associated progesterone receptor comp...,False,0.041,Membrane-associated progesterone receptor comp...,Homo sapiens (Human),23818,Alternative splicing . Endoplasmic reticulum ....,NaN,"cellular organisms , Eukaryota , Opisthokonta ...",106.81
19602,MTLPASLKGFETLDIFNISSVRNICELNKHAEHVSIKNLPLIDISV...,1,256,Cytoplasmic envelopment protein 1,False,0.916,Cytoplasmic envelopment protein 1,Human herpesvirus 7 (strain JI) (HHV-7) (Human...,29375,Host cytoplasm . Host Golgi apparatus . Refere...,Homo sapiens (Human) [TaxID: 9606],"Viruses , Duplodnaviria , Heunggongvirae (king...",114.75
19603,MKWKALFTAAILQAQLPITEAQSFGLLDPKLCYLLDGILFIYGVIL...,0,164,T-cell surface glycoprotein CD3 zeta chain,False,0.089,T-cell surface glycoprotein CD3 zeta chain (T-...,Homo sapiens (Human),18696,3D-structure . Adaptive immunity . Alternative...,NaN,"cellular organisms , Eukaryota , Opisthokonta ...",114.00
19604,MAGVGDAAAPGEGGGGGVDGPQRDGRGEAEQPGGSGGQGPPPAPQL...,0,567,Transmembrane anterior posterior transformatio...,False,0.047,Transmembrane anterior posterior transformatio...,Homo sapiens (Human),64260,Acetylation . Alternative splicing . Cell proj...,NaN,"cellular organisms , Eukaryota , Opisthokonta ...",113.33
